In [27]:
import numpy as np
import pandas as pd
import pytz
import datetime
import tqdm

In [2]:
data = pd.read_csv('minuteData_NbTicksMin25_BusDay100_2020-12-31.csv')
data.head()

,timeStamp,TSLA_open,AAPL_open,NVDA_open,ORCL_open,SPY_open,MSFT_open,C_open,FB_open,QQQ_open,...,INTC_open,PFE_open,BAC_open,NFLX_open,MU_open,XOM_open,GILD_open,SNAP_open,GE_open,IWM_open
0,2020-08-14 09:30:00-04:00,332.9980,114.8288,461.2000,54.030,336.410,208.76,52.450,262.3050,273.01,...,48.730,38.0100,26.0600,482.820,45.600,42.6000,68.090,21.9000,6.5400,156.28
1,2020-08-14 09:31:00-04:00,330.6161,114.7500,461.2800,53.865,336.420,208.93,52.565,261.9600,273.03,...,48.800,38.0600,26.1200,483.250,45.550,42.6608,68.170,21.9901,6.5300,156.47
2,2020-08-14 09:32:00-04:00,331.2920,114.6325,461.8016,53.810,336.160,208.40,52.700,262.1600,272.64,...,48.780,37.9900,26.1973,482.760,45.495,42.7500,68.120,21.9900,6.5500,156.50
3,2020-08-14 09:33:00-04:00,332.9500,114.6575,463.7550,53.890,336.415,208.92,52.560,262.3400,273.04,...,48.870,37.9618,26.1950,484.705,45.470,42.9100,68.255,22.0250,6.5499,156.67
4,2020-08-14 09:34:00-04:00,331.6272,114.4225,462.0636,53.960,336.130,208.93,52.470,261.8229,272.80,...,48.795,38.0000,26.1300,485.080,45.340,42.8300,68.200,22.0150,6.5450,156.42


In [3]:
type(data['timeStamp'][0])

str

In [18]:
data['timeStamp'] = pd.to_datetime(data['timeStamp'])

In [20]:
data.head()

,timeStamp,TSLA_open,AAPL_open,NVDA_open,ORCL_open,SPY_open,MSFT_open,C_open,FB_open,QQQ_open,...,INTC_open,PFE_open,BAC_open,NFLX_open,MU_open,XOM_open,GILD_open,SNAP_open,GE_open,IWM_open
0,2020-08-14 09:30:00-04:00,332.9980,114.8288,461.2000,54.030,336.410,208.76,52.450,262.3050,273.01,...,48.730,38.0100,26.0600,482.820,45.600,42.6000,68.090,21.9000,6.5400,156.28
1,2020-08-14 09:31:00-04:00,330.6161,114.7500,461.2800,53.865,336.420,208.93,52.565,261.9600,273.03,...,48.800,38.0600,26.1200,483.250,45.550,42.6608,68.170,21.9901,6.5300,156.47
2,2020-08-14 09:32:00-04:00,331.2920,114.6325,461.8016,53.810,336.160,208.40,52.700,262.1600,272.64,...,48.780,37.9900,26.1973,482.760,45.495,42.7500,68.120,21.9900,6.5500,156.50
3,2020-08-14 09:33:00-04:00,332.9500,114.6575,463.7550,53.890,336.415,208.92,52.560,262.3400,273.04,...,48.870,37.9618,26.1950,484.705,45.470,42.9100,68.255,22.0250,6.5499,156.67
4,2020-08-14 09:34:00-04:00,331.6272,114.4225,462.0636,53.960,336.130,208.93,52.470,261.8229,272.80,...,48.795,38.0000,26.1300,485.080,45.340,42.8300,68.200,22.0150,6.5450,156.42


0.8491678319831647

In [22]:
cols = data.columns

In [24]:
cols = cols.drop("timeStamp")
cols

Index(['TSLA_open', 'AAPL_open', 'NVDA_open', 'ORCL_open', 'SPY_open',
       'MSFT_open', 'C_open', 'FB_open', 'QQQ_open', 'AMZN_open', 'WFC_open',
       'SQQQ_open', 'WMT_open', 'UBER_open', 'BA_open', 'INTC_open',
       'PFE_open', 'BAC_open', 'NFLX_open', 'MU_open', 'XOM_open', 'GILD_open',
       'SNAP_open', 'GE_open', 'IWM_open'],
      dtype='object')

In [25]:
cols = [x[:-5] for x in cols]
cols

['TSLA',
 'AAPL',
 'NVDA',
 'ORCL',
 'SPY',
 'MSFT',
 'C',
 'FB',
 'QQQ',
 'AMZN',
 'WFC',
 'SQQQ',
 'WMT',
 'UBER',
 'BA',
 'INTC',
 'PFE',
 'BAC',
 'NFLX',
 'MU',
 'XOM',
 'GILD',
 'SNAP',
 'GE',
 'IWM']

In [26]:
log_returns = pd.DataFrame(columns = [x+'_log_return' for x in cols])
vol_5 = pd.DataFrame(columns = [x+'_vol_5' for x in cols])
vol_30 = pd.DataFrame(columns = [x+'_vol_30' for x in cols])

In [35]:
for index, row in tqdm.tqdm(data.iterrows(), total = data.shape[0]):
    log_return = []
    v_5 = []
    v_30 = []
    for c in cols:
        if index > 0:
            log_return.append(np.log(data[c+'_open'][index]/data[c+'_open'][index-1]))
        if index > 3:
            v_5.append(np.std(data[c+'_open'][index-4:index+1]))
        if index > 28:
            v_30.append(np.std(data[c+'_open'][index-29:index+1])) 
            
    if index>0:
        log_returns.loc[len(log_returns.index)] = log_return
    if index>3:
        vol_5.loc[len(vol_5.index)] = v_5
    if index>28:
        vol_30.loc[len(vol_30.index)] = v_30
            


  1%|          | 336/37470 [00:13<31:39, 19.55it/s]

KeyboardInterrupt: 

In [31]:
log_returns.head()

,TSLA_log_return,AAPL_log_return,NVDA_log_return,ORCL_log_return,SPY_log_return,MSFT_log_return,C_log_return,FB_log_return,QQQ_log_return,AMZN_log_return,...,INTC_log_return,PFE_log_return,BAC_log_return,NFLX_log_return,MU_log_return,XOM_log_return,GILD_log_return,SNAP_log_return,GE_log_return,IWM_log_return
0,-0.007179,-0.000686,0.000173,-0.003059,0.000030,0.000814,0.002190,-0.001316,0.000073,-0.000371,...,0.001435,0.001315,0.002300,0.000890,-0.001097,0.001426,0.001174,0.004106,-0.001530,0.001215
1,0.002042,-0.001024,0.001130,-0.001022,-0.000773,-0.002540,0.002565,0.000763,-0.001429,-0.003007,...,-0.000410,-0.001841,0.002955,-0.001014,-0.001208,0.002089,-0.000734,-0.000005,0.003058,0.000192
2,0.004992,0.000218,0.004221,0.001486,0.000758,0.002492,-0.002660,0.000686,0.001466,0.001511,...,0.001843,-0.000743,-0.000088,0.004021,-0.000550,0.003736,0.001980,0.001590,-0.000015,0.001086
3,-0.003981,-0.002052,-0.003654,0.001298,-0.000848,0.000048,-0.001714,-0.001973,-0.000879,0.000548,...,-0.001536,0.001006,-0.002484,0.000773,-0.002863,-0.001866,-0.000806,-0.000454,-0.000748,-0.001597
4,-0.006071,-0.002253,-0.003607,0.001481,-0.000268,-0.002204,0.001333,-0.002114,-0.001394,-0.002560,...,0.000000,0.000263,0.001530,-0.000144,0.000441,0.001633,0.000440,-0.005237,0.001527,-0.000703


In [36]:
vol_5.head()

,TSLA_vol_5,AAPL_vol_5,NVDA_vol_5,ORCL_vol_5,SPY_vol_5,MSFT_vol_5,C_vol_5,FB_vol_5,QQQ_vol_5,AMZN_vol_5,...,INTC_vol_5,PFE_vol_5,BAC_vol_5,NFLX_vol_5,MU_vol_5,XOM_vol_5,GILD_vol_5,SNAP_vol_5,GE_vol_5,IWM_vol_5
0,0.938140,0.136928,0.925184,0.076577,0.132650,0.204490,0.088566,0.199030,0.158821,3.795636,...,0.044944,0.032138,0.051395,0.976963,0.087886,0.111764,0.058275,0.044207,0.007465,0.126079
1,1.103502,0.209687,1.104888,0.079725,0.155743,0.241909,0.074673,0.365263,0.236440,4.118024,...,0.031718,0.032138,0.032206,0.964782,0.080474,0.094192,0.047117,0.044207,0.008593,0.117405
2,1.086445,0.186601,1.154128,0.081731,0.131286,0.256796,0.086163,0.368218,0.211811,3.001680,...,0.031718,0.022565,0.027596,1.018570,0.061612,0.061838,0.055281,0.070498,0.003737,0.129213
3,1.168755,0.161923,1.267422,0.063056,0.142632,0.260875,0.043081,0.401022,0.212941,2.787959,...,0.029563,0.042128,0.023537,0.621292,0.066603,0.070711,0.048415,0.072553,0.008355,0.170341
4,0.836973,0.089451,1.284964,0.058310,0.170529,0.301423,0.032619,0.378062,0.233889,2.624043,...,0.010992,0.036551,0.018868,0.705141,0.077720,0.059174,0.058515,0.061205,0.009517,0.103034



  1%|          | 336/37470 [00:32<31:39, 19.55it/s]

In [37]:
vol_30.head()

,TSLA_vol_30,AAPL_vol_30,NVDA_vol_30,ORCL_vol_30,SPY_vol_30,MSFT_vol_30,C_vol_30,FB_vol_30,QQQ_vol_30,AMZN_vol_30,...,INTC_vol_30,PFE_vol_30,BAC_vol_30,NFLX_vol_30,MU_vol_30,XOM_vol_30,GILD_vol_30,SNAP_vol_30,GE_vol_30,IWM_vol_30
0,1.305705,0.362550,1.778925,0.104934,0.212408,0.365969,0.060673,0.643585,0.430724,8.776870,...,0.062904,0.046381,0.036379,1.626095,0.135028,0.097937,0.076545,0.089688,0.013265,0.210908
1,1.241806,0.341990,1.691428,0.106750,0.220761,0.369471,0.059500,0.631061,0.426226,8.516449,...,0.061138,0.049667,0.030824,1.578571,0.135235,0.090687,0.080504,0.089587,0.012936,0.229076
2,1.260227,0.321087,1.619430,0.101066,0.236867,0.369196,0.058914,0.627104,0.421093,8.380171,...,0.065239,0.055480,0.029158,1.560141,0.135335,0.085950,0.086243,0.087067,0.011806,0.241550
3,1.255219,0.304867,1.549451,0.086599,0.240861,0.364865,0.051651,0.616619,0.421253,8.697911,...,0.066915,0.058395,0.029483,1.472857,0.134078,0.085730,0.088502,0.084357,0.012697,0.252277
4,1.181466,0.284351,1.551113,0.079208,0.249831,0.365820,0.053572,0.597901,0.415051,8.733007,...,0.071645,0.058821,0.029692,1.460369,0.132858,0.088217,0.090930,0.079366,0.014144,0.258733
